In [ ]:
import feature_engineering as fe
import train_utils as tu
from persistent_cache import memo, PersistentDict as Perd
from sklearn.cross_validation import StratifiedKFold, KFold

In [14]:
import numpy as np
from sklearn.cross_validation import StratifiedKFold, KFold
from train_utils import info, linreg, xgbr
from train_utils import benchmark_model_optimized
from joblib import Memory

def mini_train_predictions(model, X, y, train_test_folds):
    ind2pred = {}
    for i, (train, test) in enumerate(train_test_folds):
        info(("fitting MINIfold   "+str(i+1)+ str(model)[:100]))
        model.fit(X[train], y[train])
        info(("MINIfold fitted    "+str(i+1)+  str(model)[:100]))
        preds = model.predict(X[test])
        for i, p in zip(test, preds):
            ind2pred[i] = p
    
    return np.array([ind2pred[i] for i in range(len(y))])

class Stacker(object):
    def __init__(self, stacker, base_models, folds=4):
        self.folds = folds
        self.base_models = sorted(base_models)
        self.stacker = stacker
        
    def fit(self, X, y):
        folds = list(StratifiedKFold(y, n_folds=self.folds, random_state=0))
        base_preds = [mini_train_predictions(m, X, y, folds) for m in self.base_models]
        n = len(y)
        XX = np.hstack([X] + [p.reshape(n, 1) for p in base_preds])
        print "fit", XX.shape
        self.stacker.fit(XX, y)
        for m in self.base_models:
            m.fit(X, y)
        return self
    
    def predict(self, X):
        n = X.shape[0]
        XX = np.hstack([X] + [m.predict(X).reshape(n, 1) for m in self.base_models])
        print "predict", XX.shape
        return self.stacker.predict(XX)

In [17]:
model = Stacker(linreg(), [linreg()], 2)

In [18]:
benchmark_model_optimized(model, "basic")

INFO:trainlo:optimising offsets 44533
INFO:trainlo:done optimising offsets 44533
INFO:trainlo:optimising offsets 44536
INFO:trainlo:done optimising offsets 44536
INFO:trainlo:optimising offsets 44536
INFO:trainlo:done optimising offsets 44536
INFO:trainlo:optimising offsets 44538
INFO:trainlo:done optimising offsets 44538
INFO:trainlo:optimized 0.5958   <__main__.Stacker object at 0x7f8002385b10>  basic


0.5958324742368268

In [ ]:
lin = LinearRegression()

In [ ]:
X, XX = fe.basic_extractor()

In [ ]:
X.shape